In [1]:
from Adafruit_IO import Client, Feed
from i2c_sensor_classes import Ky015Sensor, Ky018Sensor
from aio_helper_classes import HabitatMonitorClient
import time
import threading
import asyncio

In [2]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

# Testing Reading from the Sensors 
no cloud here, trying to decouple cloud code and sensor reading code

In [ ]:
KY015_ADDR = 8
KY018_ADDR = 0x28
SCL_PIN = 2
SDA_PIN = 3

liba = MicroblazeLibrary(base.PMODA, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

libb = MicroblazeLibrary(base.PMODB, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky018 = Ky018Sensor(libb, SDA_PIN, SCL_PIN, KY018_ADDR)

response = input('Read Data?(y/n): ')
while response != 'n':
    temp, hum = ky015.get_temp_and_hum_data()
    bright = ky018.get_brightness_data()
    print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
    response = input('Read Data?(y/n): ')

# Testing the Habitat Monitor class
this class extends the default Adafruit_IO Client, with functionality specific to out Habitat Monitor. If check the client_id and either sets the feeds to the correct Feed Objects, or if the client_id is new then new data feeds are created and associated with the passed client-id. 

In [ ]:
UNAME = input('Enter username: ')
KEY = input('Enter Key: ')
client_id = 'ryans-pynq-board'

#custom class see "src/aio_helper_classes.py"
aio = HabitatMonitorClient(UNAME, KEY, client_id)    

In [ ]:
response = 'y'
while response != 'n':
    response = input('Read Data?(y/n): ')
    if response == 'n': break
    temp, hum = ky015.get_temp_and_hum_data()
    bright = ky018.get_brightness_data()
    print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
    response = input('Publish to Cloud?(y/n): ')
    if response == 'n': break
    aio.send(aio.temperature_feed.key, temp)
    aio.send(aio.humidity_feed.key, hum)
    aio.send(aio.brightness_feed.key, bright)
    print('\tData Sent to Adafrui.io')

# Client Code
## ToDo
- Integrate Threading + LED + Button code from Ryan
- Need to reace to thresholds and blink leds
- Should be able to receive new threshold values from cloud
- should Read AIO uname + key from a local file (dont push the file to git)
- launch all client threads in the client_process fucniton 

## Threads for Client

In [14]:
def client_button(btns):
    global stop
    while not stop:
        if btns.read() == 0x01:
            stop = True
    print("Button was pressed! Ending program...")
    
def client_LED(LEDStates, LEDLock):
    #Process to handle blinking the LEDs to show various statuses on the client board
    blinkrate = 1
    t_on = 1/(2*blinkrate)
    
    blinkOn = False; #want lights to blink in sync, this stores state
    
    while not stop:
        usingLock = LEDLock.acquire(True)
        if usingLock:
            for i in range(4):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.leds[i].off()
                    else:
                        base.leds[i].on()

            for i in range(4,6):
                if LEDStates[i] == 1:
                    if blinkOn:
                        base.rgbleds[i].off()
                    else:
                        base.rgbleds[i].on(0b100)
            LEDLock.release()
        if blinkOn:
            blinkOn = False
        else:
            blinkOn = True
                 
        time.sleep(t_on)
    
def start_blink(LEDStates,index):
    LEDStates[index] = 1
    
def stop_blink(LEDStates,index):
    LEDStates[index] = 0
    led_off(LEDStates,index)
    
def blinkAllLEDs():
    for i in LEDStates:
        i = 1
    
def led_off(LEDStates,index):
    if index < 0 or index > 6:
        return
    elif(LEDStates[index] == 1):
        return
    elif index < 4:
        base.leds[index].off()
    else:
        base.rgbleds[index].off()
        
def led_on(LEDStates,index):
    if index < 0 or index > 6:
        return
    elif(LEDStates[index] == 1):
        return
    elif index < 4:
        base.leds[index].on()
    else:
        base.rgbleds[index].on(0b100)

In [15]:
def client_sensors(aio,LEDStates,LEDLock):
    asyncio.set_event_loop(asyncio.new_event_loop()) #required to avoid no event loop error
    
    KY015_ADDR = 8
    KY018_ADDR = 0x28
    SCL_PIN = 2
    SDA_PIN = 3

    liba = MicroblazeLibrary(base.PMODA, ['i2c'])
    #custom class see 'src/i2c_sensor_classes.py'
    ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

    libb = MicroblazeLibrary(base.PMODB, ['i2c'])
    #custom class see 'src/i2c_sensor_classes.py'
    ky018 = Ky018Sensor(libb, SDA_PIN, SCL_PIN, KY018_ADDR)

    lbtFeed = aio.feeds('low-brightness-threshold')
    hbtFeed = aio.feeds('high-brightness-threshold')
    lttFeed = aio.feeds('low-temperature-threshold')
    httFeed = aio.feeds('high-temperature-threshold')
    lhtFeed = aio.feeds('low-humidity-threshold')
    hhtFeed = aio.feeds('high-humidity-threshold')
    
    tempIsGood = True
    thresholdsUpToDate = True
    samplectr = 0
    numberOfFailedPushes = 0
    while not stop:
        start_time = time.time()
        if samplectr == 0:
            led_on(LEDStates,2) #solid LED 2 indicates updating thresholds
            try:
                [lowB, highB] = updateBrightnessThresholds(aio,lbtFeed,hbtFeed)
                [lowT, highT] = updateTemperatureThresholds(aio,lttFeed,httFeed)
                [lowH, highH] = updateHumidityThresholds(aio,lhtFeed,hhtFeed)
                if not thresholdsUpToDate:
                    thresholdsUpToDate = True
                    usinglock = LEDLock.acquire(True)
                    if usinglock:
                        stop_blink(LEDStates,2)
                        LEDLock.release()
            except:
                thresholdsUpToDate = False
                print("Error updating thresholds")
                start_blink(LEDStates,2) #blinking LED 2 indicates threshold may be out of date
            led_off(LEDStates,2)
        samplectr = (samplectr + 1) % SAMPLES_BEFORE_THRESHOLD_UPDATE
        
        temp, hum = ky015.get_temp_and_hum_data()
        bright = ky018.get_brightness_data()
        print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
        
        #Temperature check
        if temp < lowT:
            tempIsGood = False
            start_blink(LEDStates,4)
            print("ALERT: Temperature is lower than threshold!")
        elif temp > highT:
            tempIsGood = False
            start_blink(LEDStates,4)
            print("ALERT: Temperature is higher than threshold!")
        else:
            tempIsGood = True
        
        #Humidity check
        if hum < lowH:
            start_blink(LEDStates,4)
            print("ALERT: Humidity is lower than threshold!")
        elif hum > highH:
            start_blink(LEDStates,4)
            print("ALERT: Humidity is higher than threshold!")
        else:
            if tempIsGood:
                #temp and humidity are both good, can turn off that LED
                usinglock = LEDLock.acquire(True)
                if usinglock:
                    stop_blink(LEDStates,4)
                    LEDLock.release()
        
        #Brightness check
        if bright < lowB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is lower than threshold!")
        elif bright > highB:
            start_blink(LEDStates,5)
            print("ALERT: Brightness is higher than threshold!")
        else:
            usinglock = LEDLock.acquire(True)
            if usinglock:
                stop_blink(LEDStates,5)
                LEDLock.release()
        
        
        try:
            aio.send(aio.temperature_feed.key, temp)
            aio.send(aio.humidity_feed.key, hum)
            aio.send(aio.brightness_feed.key, bright)
            print('\tData Sent to Adafruit.io')
            if numberOfFailedPushes > 0:
                numberOfFailedPushes = 0
                usinglock = LEDLock.acquire(True)
                if usinglock:
                    stop_blink(LEDStates,0)
                    stop_blink(LEDStates,1)
                    LEDLock.release()
            led_on(LEDStates,3)
            time.sleep(0.3) #A blink on LED 3 indicates a successful push to the cloud
            led_off(LEDStates,3)
        except:
            print("Error pushing data to adafruit.io")
            numberOfFailedPushes += 1
            if numberOfFailedPushes >= 5:
                start_blink(LEDStates,0)
                start_blink(LEDStates,1)
        
        wake_time = start_time + SAMPLE_PERIOD
        sleep_time = wake_time - time.time()
        if (sleep_time > 0):
            time.sleep(sleep_time)
    
def updateBrightnessThresholds(aio,lbtFeed,hbtFeed):
    lowB = int(aio.receive(lbtFeed.key).value)
    highB = int(aio.receive(hbtFeed.key).value)
    print("Brightness threshold updated: {} to {}".format(lowB,highB))
    return [lowB,highB]

def updateTemperatureThresholds(aio,lttFeed,httFeed):
    lowT = float(aio.receive(lttFeed.key).value)
    highT = float(aio.receive(httFeed.key).value)
    print("Temperature threshold updated: {}C to {}C".format(lowT,highT))
    return [lowT,highT]

def updateHumidityThresholds(aio,lhtFeed,hhtFeed):
    lowH = float(aio.receive(lhtFeed.key).value)
    highH = float(aio.receive(hhtFeed.key).value)
    print("Humidity threshold updated: {}% to {}%".format(lowH,highH))
    return [lowH,highH]

In [18]:
def client_process():
    
    with open('adafruitkey.txt') as file:
        UNAME = file.readline().rstrip('\n')
        KEY = file.readline().rstrip('\n')
        
    client_id = 'ryans-pynq-board'
    
    global stop
    global SAMPLE_PERIOD 
    SAMPLE_PERIOD = 2 #number of seconds to wait between each sample
    global SAMPLES_BEFORE_THRESHOLD_UPDATE
    SAMPLES_BEFORE_THRESHOLD_UPDATE = 5  #number of samples to take before updating threshold again

    #custom class see "src/aio_helper_classes.py"
    aio = HabitatMonitorClient(UNAME, KEY, client_id)

    stop = False

    LEDStates = [0, 0, 0, 0, 0, 0] #store states of LEDs. 0 = steady, 1 = blinking

    LEDLock = threading.Lock()

    btns = base.btns_gpio

    t1 = threading.Thread(target=client_sensors, args=(aio,LEDStates,LEDLock))
    t1.start()
    t2 = threading.Thread(target=client_LED, args=(LEDStates,LEDLock)) 
    t2.start()
    t3 = threading.Thread(target=client_button, args=(btns,))
    t3.start()
    t1.join()
    print('t1 joined')
    t2.join()
    print('t2 joined')
    t3.join()
    print('t3 joined')
    print('Client process exited.')

In [19]:
client_process()

creating new HabitatMonitor...
HabitatMonitorClientsetup
Brightness threshold updated: 410 to 1205
Temperature threshold updated: 11.0C to 63.0C
Humidity threshold updated: 27.0% to 64.0%
	Temp: 20.04C, Hum: 50.0%
	Bright: 716
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 706
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 715
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 713
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 700
	Data Sent to Adafruit.io
Brightness threshold updated: 410 to 1205
Temperature threshold updated: 11.0C to 63.0C
Humidity threshold updated: 27.0% to 64.0%
	Temp: 20.04C, Hum: 50.0%
	Bright: 92
ALERT: Brightness is lower than threshold!
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 84
ALERT: Brightness is lower than threshold!
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 646
	Data Sent to Adafruit.io
	Temp: 20.04C, Hum: 50.0%
	Bright: 546
Button was pressed! Ending program...


# Server Code
## ToDo
Create a server that can monitor the cloud feeds and change thresholds

In [ ]:
def server_process():
    pass

# main() Function
## ToDo
- Launch client process 
- Optionally launch server process

In [ ]:
def main():
    pass